In [1]:
import sys
sys.path.append("..")
import ScraperFC as sfc # import local scraperfc
import traceback

In [2]:
scraper = sfc.FBRef()
links = [
    "https://fbref.com/en/matches/1b2886ce/Qatar-Ecuador-November-20-2022-World-Cup",
    "https://fbref.com/en/matches/e62f6e78/Crystal-Palace-Arsenal-August-5-2022-Premier-League",
    "https://fbref.com/en/matches/3cc3f661/Los-Angeles-FC-Colorado-Rapids-February-26-2022-Major-League-Soccer",
    "https://fbref.com/en/matches/0804c2fc/CF-Montreal-New-York-City-FC-October-23-2022-Major-League-Soccer",
    "https://fbref.com/en/matches/e38a53be/Los-Angeles-FC-Philadelphia-Union-November-5-2022-Major-League-Soccer",
    # "https://fbref.com/en/matches/07f058d4/Dinamo-Zagreb-Chelsea-September-6-2022-Champions-League",
]
try:
    for link in links:
        print(link)
        match = scraper.scrape_match(link)
        print(match.shape)
except:
    traceback.print_exc()
finally:
    scraper.close()



====== WebDriver manager ======
Current google-chrome version is 107.0.5304
Get LATEST chromedriver version for 107.0.5304 google-chrome
Driver [C:\Users\oseymour\.wdm\drivers\chromedriver\win32\107.0.5304.62\chromedriver.exe] found in cache


https://fbref.com/en/matches/1b2886ce/Qatar-Ecuador-November-20-2022-World-Cup
(14, 28) 14
(16, 28) 16
(1, 22)
https://fbref.com/en/matches/e62f6e78/Crystal-Palace-Arsenal-August-5-2022-Premier-League
(16, 29) 16
(15, 29) 15
(1, 22)
https://fbref.com/en/matches/3cc3f661/Los-Angeles-FC-Colorado-Rapids-February-26-2022-Major-League-Soccer
(17, 29) 17
(16, 29) 16
(1, 22)
https://fbref.com/en/matches/0804c2fc/CF-Montreal-New-York-City-FC-October-23-2022-Major-League-Soccer
(17, 29) 17
(16, 29) 16
(1, 22)
https://fbref.com/en/matches/e38a53be/Los-Angeles-FC-Philadelphia-Union-November-5-2022-Major-League-Soccer
(16, 29) 16
(15, 29) 15
(1, 22)


In [3]:
match

,Link,Date,Stage,Home Team,Away Team,Home Team ID,Away Team ID,Home Formation,Away Formation,Home Goals,...,Away Ast,Home xG,Away xG,Home npxG,Away npxG,Home xAG,Away xAG,Home Player Stats,Away Player Stats,Shots
0,https://fbref.com/en/matches/e38a53be/Los-Ange...,2022-11-05,MLS Cup,Los Angeles FC,Philadelphia Union,81d817a3,46024eeb,4-3-3,4-3-1-2,3,...,2,1.0,1.6,1.0,1.6,0.9,0.6,Team Sheet Los Angeles FC (4-3-3) L...,Team Sheet Philadelphia (4-3-1-2) Philad...,Both Unnamed: 0_level_0 Unnam...
